### 네이버 검색량과 연령대별 TV 광고 시청자 수 비교
- 사용 데이터
    - TV GRPs by models
    - naver query

In [1]:
import pandas as pd
import matplotlib
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from sklearn import preprocessing

pd.options.plotting.backend = 'plotly'
pio.renderers.default = "notebook_connected"

In [2]:
query_data = pd.read_csv('./datas/naver_query.csv')
tv_grps_data = pd.read_csv('./datas/TV_GRPs_v1.csv')

query_data = query_data.set_index('date')
model_df = query_data.drop(['Year', 'Month'], axis=1)
model_df.head(3)

,Mercedes-Benz,BMW,MINI,Audi,Volkwagen,Porsche,Toyota,Lexus,Nissan,Infiniti,...,Land Rover,Ford,GM,Cadillac,Chrysler,Fiat,Maserati,Peugeot,Citroen,Aston Martin
date,,,,,,,,,,,,,,,,,,,,,
2016-01-01,91367.5,36268.9,1014.9,50761.1,12804.6,11505.2,6687.9,5574.7,1400.5,13663.1,...,40962.2,9505.0,605.4,12082.1,9170.2,954.6,15607.8,6558.9,348.1,33641.6
2016-01-02,94927.9,39025.6,1132.1,52742.7,13012.0,11648.4,6821.0,6028.9,1585.1,14435.0,...,41032.9,9338.6,644.8,9895.6,9670.6,1129.1,15573.4,6868.3,333.9,12209.4
2016-01-03,93701.7,39610.2,1085.6,54485.8,13314.5,10978.5,6840.0,5785.8,1798.1,14423.9,...,38500.1,10186.1,694.2,10135.7,9399.1,2208.8,13446.5,6681.6,358.1,5497.2


### 네이버 검색량 추이 비교
- 2016 ~ 2021년까지 네이버 검색량 추이 비교
- 벤츠는 증가, 상승 폭이 크지 않고 비교적 꾸준히 상위권을 유지하는 경향을 보인다.
- 특정 시기에 검색량이 대폭 상승하는 경우가 있는데 , 부정적인 이슈인지 긍정적인 이슈인지 추가적으로 확인할 필요가 있어보인다.

In [3]:
fig = px.line(model_df, x=model_df.index, y=model_df.columns)

fig.update_layout(font_size=14, width=1000, height=500, template="plotly_white")
fig.update_annotations(font_size=19)
fig.show()

In [4]:
# 벤츠는 연도별 추세를 봤을 때 꾸준히 인기가 있는 것 같다.
# 월별로 확인
month_df = query_data.drop(['Year'], axis=1)
month_df = month_df.groupby('Month').mean().round(3)
# month_df = month_df.sort_values(ascending=False, by=month_df.sum())
month_df.head(3)

,Mercedes-Benz,BMW,MINI,Audi,Volkwagen,Porsche,Toyota,Lexus,Nissan,Infiniti,...,Land Rover,Ford,GM,Cadillac,Chrysler,Fiat,Maserati,Peugeot,Citroen,Aston Martin
Month,,,,,,,,,,,,,,,,,,,,,
1,124467.293,55949.374,1188.556,47565.768,13103.222,14233.370,8832.648,7293.968,2213.467,18340.083,...,40806.858,13540.106,1099.202,23809.801,11046.070,2128.700,19419.152,5688.309,1157.356,7662.971
2,133376.394,59206.624,1240.053,49309.506,13972.631,15328.775,9078.014,7726.351,2193.258,18997.623,...,43194.059,13402.241,2254.964,25378.596,11385.574,2189.875,18970.418,6157.096,1064.363,4405.846
3,127600.848,58146.068,1290.897,49928.805,12611.731,17830.950,9755.346,7257.866,2178.227,19191.439,...,42807.120,11914.047,1598.371,25791.208,12987.836,1858.661,17799.007,5697.324,988.603,3089.995


### 월별 브랜드별 평균 비중
- 브랜드별 월별 평균 값으로 비중을 확인했을 때 벤츠가 검색량 압도적 1위를 차지.
- 벤츠를 이어 BMW가 2위를 차지. 
- BMW가 2위지만 BMW와 벤츠의 검색량이 두배 가까이 차이나므로 검색량만 봤을 때 경쟁 구도로 볼 수 있을지 모르겠다. 

In [5]:
# 월별 브랜드별 평균 비중 확인
fig = go.Figure()

for i in range(len(month_df.columns)):
    fig.add_trace(
        go.Bar(name=month_df.columns[i], y=month_df.iloc[:,i], x=month_df.index))

fig.update_layout(barmode='stack')
fig.show()

### 벤츠 월별 네이버 쿼리량과 연령별 월별 평균 tv 광고 시청자 수 비교
- 스케일러를 적용하기 전에는 tv 광고 시청자 수 (특히 50,60대를 중심으로)가 월등히 많아 추이 비교 불가
- min-max scaler 적용
    - 연령별 tv 시청 추이와 비교했을 때 비슷한 양상을 보이는지는 모르겠다..
    - 네이버 검색 쿼리 데이터는 연령별로 구분되어 있지 않아 연령별로 구분된 데이터가 있다면 더 자세히 비교해 볼 수 있을 것 같다.

In [7]:
tv_grps_data = pd.read_csv('./datas/TV_GRPs_v1.csv')
tv_grps_data = tv_grps_data.dropna()

tv_grps_data['Date'] = tv_grps_data['Date'].astype(int).astype(str)
tv_grps_data = tv_grps_data.set_index('Date')
tv_benz_df = tv_grps_data[tv_grps_data['Advertiser'] == '메르세데스벤츠코리아']

query_data = pd.read_csv('./datas/naver_query.csv')
query_data['Date'] = query_data['date']
query_data['Year'] = query_data['Year'].astype('str')
query_data['Month'] = query_data['Month'].astype('str')
query_data = query_data[['Mercedes-Benz', 'Date']]

query_data['Date'] = query_data['Date'].str.slice(stop=7)
query_data['Date'] = query_data['Date'].str.replace('-','')
query_data = query_data.set_index('Date')

query_gb = query_data.groupby(query_data.index).mean().round(3)

reset_query_gb = query_gb.reset_index()
reset_query_gb

reset_tv_benz = tv_benz_df[['Viewers_20s','Viewers_30s', 'Viewers_40s', 'Viewers_50s', 'Viewers_60s']].reset_index()

# min-max scaler
scaled_df = pd.DataFrame(preprocessing.minmax_scale(concat_df)).round(3)
scaled_df.columns = concat_df.columns.tolist()
scaled_df['Date'] = concat_df['Date']

# 벤츠 월별 네이버 쿼리량과 연령별 연도별 월별 평균 tv 광고 시청자 수
concat_df = pd.merge(reset_query_gb, reset_tv_benz, on='Date')
concat_df

NameError: name 'concat_df' is not defined

In [9]:
fig = make_subplots(rows=1, cols=1)
ages = ['20', '30', '40', '50', '60']

for age in ages:
    fig.add_trace(go.Scatter(x=scaled_df.Date, 
                             y=scaled_df[f'Viewers_{age}s'], 
                             name=f'Viewers{age}s',
                             line=dict(width=2.5)))
    
fig.add_trace(go.Scatter(x=scaled_df.Date, 
                         y=scaled_df['Mercedes-Benz'],
                         name='Benz query',
                         line=dict(width=4, dash='dash')))  
    
fig.update_layout(font_size=14, width=1500, height=500, template="plotly_white")
fig.update_annotations(font_size=19)

fig.show()

NameError: name 'scaled_df' is not defined

### 느낀점
- 벤츠와 비슷한 추세를 보이는 브랜드와 함께 비교해보려고 했으나, 벤츠가 압도적
- 생각보다 연도에 따른, 월별에 따른 변동이 크지 않다.
- 당연한 결과지만 벤츠 tv 광고 시청자수가 네이버 검색자 수보다 월등히 많았다. (50, 60대가 압도적으로..)